In [2]:
#Import librarys
import os

import torch
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.io import read_image
from torchvision.transforms import ToTensor

import torchvision.transforms as transforms

from torch.utils.data import Dataset

import numpy as np

In [3]:
#Locale where the imagens stay
image_path = 'images/'

#Getting the list of classes (considering each fold at "image_path" like a class)
list_classes = [f for f in os.listdir(image_path)]

num_classes = len(list_classes)

img_size = 224

In [4]:
# def load_split_train_test(datadir, valid_size = .2):
#     train_transforms = transforms.Compose([transforms.Resize(64),
#                                        transforms.ToTensor(),
#                                        ])
#     test_transforms = transforms.Compose([transforms.Resize(64),
#                                       transforms.ToTensor(),
#                                       ])
#     train_data = datasets.ImageFolder(datadir,       
#                     transform=train_transforms)
#     test_data = datasets.ImageFolder(datadir,
#                     transform=test_transforms)
#     num_train = len(train_data)
#     indices = list(range(num_train))
#     split = int(np.floor(valid_size * num_train))
#     np.random.shuffle(indices)
#     from torch.utils.data.sampler import SubsetRandomSampler
#     train_idx, test_idx = indices[split:], indices[:split]
#     train_sampler = SubsetRandomSampler(train_idx)
#     test_sampler = SubsetRandomSampler(test_idx)
#     trainloader = torch.utils.data.DataLoader(train_data,
#                    sampler=train_sampler, batch_size=64)
#     testloader = torch.utils.data.DataLoader(test_data,
#                    sampler=test_sampler, batch_size=64)
    
#     return trainloader, testloader

# trainloader, testloader = load_split_train_test(image_path, .2)
# print(trainloader.dataset.classes)

In [5]:
#Creating the class rensposable to load the imagens
class ImageDataset(Dataset):
    def __init__(self, list_classes, img_dir, transform=None, target_transform=None):
        """
        Define where the images stay, the classes of imagens and transformations that will be aplied.
        """
        #Creating the class variables
        self.list_classes = list_classes
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        
        self.imagens = []
        
        #Loop for each class
        for select_class in self.list_classes:
            #Path with original fold and class
            images_in_fold = os.listdir(f"{self.img_dir}/{select_class}/")
            #Add at the list "class name"_"image name"
            [self.imagens.append(f"{select_class}_{image}") for image in images_in_fold]

        
    def __len__(self):
        """
        Return the number of images (dataset size), this number are utilized
        in DataLoader to shuffle the data (to how how numbers can be selected).
        """
        return len(self.imagens)
    
    def __getitem__(self, idx):
        """
        Return a image and yout label.
        """
        select_sample = self.imagens[idx]
        print('idx', idx)
        select_class, image_path = select_sample.split('_')
        class_path = f"{self.img_dir}/{select_class}/"
        print('select_class, image_path ',select_class, image_path)
        
        image = read_image(f"{class_path}{image_path}")
        label = list_classes.index(select_class)
        
        if self.transform:
            #If the image hasn't the 3 channel, are them are create
            try:
                image = self.transform(image)
            except:
                print(f'Image Original Shape {image.shape}')
                image = np.stack([image, image, image])
                print(f'New Shape {image.shape}')
                image = torch.from_numpy(image)
                print(f'Converted in Tensor {image.shape}')
                image = self.transform(image)
                
        
        if self.target_transform:
            label = self.target_transform(label)
        
        return image, label

In [6]:
#Data Transfor (Normalize the image values)
transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((img_size, img_size)),
     transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

In [7]:
#Instancin the dataset (giving the fold with images and list of classes)
training_dataset = ImageDataset(list_classes, image_path)

In [8]:
#Testing the return of size the dataset
assert type(training_dataset.__len__()) == int, "The len must be a integer"

In [9]:
#Testing the return of datalaoder (two itens)
assert len(training_dataset.__getitem__(1)) == 2, "The return must by have 2 itens"

idx 1
select_class, image_path  bishopImages 00000002.jpg


In [10]:
# transform(read_image("images/bishopImages/00000206.jpg"))

## Creating a DataLoader

In [11]:
train_dataloader = DataLoader(training_dataset, batch_size=64, shuffle=True)

In [12]:
train_features, train_labels = next(iter(train_dataloader))

idx 314
select_class, image_path  KnightImages 00000302.jpg
idx 337
select_class, image_path  KnightImages 00000395.jpg
idx 205
select_class, image_path  KnightImages 00000057.jpg
idx 275
select_class, image_path  KnightImages 00000212.jpg
idx 183
select_class, image_path  KnightImages 00000024.jpg
idx 597
select_class, image_path  RookImage 00000092.jpg
idx 300
select_class, image_path  KnightImages 00000267.jpg
idx 225
select_class, image_path  KnightImages 00000089.jpg
idx 451
select_class, image_path  Queenimages 00000052.jpg
idx 656
select_class, image_path  RookImage 00000236.jpg
idx 338
select_class, image_path  PawnImages 00000001.jpg
idx 668
select_class, image_path  RookImage 00000263.jpg
idx 311
select_class, image_path  KnightImages 00000299.jpg
idx 190
select_class, image_path  KnightImages 00000033.jpg
idx 347
select_class, image_path  PawnImages 00000011.jpg
idx 124
select_class, image_path  bishopImages 00000188.jpg
idx 548
select_class, image_path  RookImage 00000016.j

RuntimeError: stack expects each tensor to be equal size, but got [4, 600, 600] at entry 0 and [3, 1024, 698] at entry 1

## Creating the Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#Define if the pytorch will use cpu or gpu to training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
# conv1 = nn.Conv2d(3, 32, 32)
# conv2 = nn.Conv2d(32, 8, 8)
# pool = nn.MaxPool2d(2, 2)

# a = pool(F.relu(conv1(train_features)))
# pool(F.relu(conv1(train_features))).shape

# pool(F.relu(conv2(a))).shape

In [ ]:
#Define the network class architecture

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 8, 8)
        self.fc1 = nn.Linear(8 * 44 * 44, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = Net()

## Define the loss function

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
model.to(device)
criterion.to(device)

## Training the network

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs.to(device))
        loss = criterion(outputs.to(device), labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2 == 0:    # print every mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
PATH = './chess_model.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
# test_images = os.listdir(f"{image_path}/{list_classes[0]}/")

# training_dataset = ImageDataset(list_classes, image_path)
# training_dataset.__len__()

## Predict image

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
model = Net()
model.load_state_dict(torch.load(PATH))
model.eval()

In [40]:
img = read_image('image.jpg')

In [ ]:
# unsqueeze provides the batch dimension
img_tensor = transform(img).to('cuda').unsqueeze(0)

In [ ]:
tensor_image = img_tensor.view(img_tensor.shape[3], img_tensor.shape[2], img_tensor.shape[1])
print(type(tensor_image), tensor_image.shape)

In [ ]:
plt.imshow(tensor_image.to('cpu'))

In [ ]:
prediction = model(img_tensor)

In [ ]:
predicted_class = np.argmax(prediction[0].to('cpu').detach().numpy())
predicted_class

In [ ]:
list_classes[1]

In [ ]:
len(list_classes)